In [1]:
import pandas as pd  
import matplotlib.pyplot as plt
import datetime, random
import numpy as np; np.random.seed(0)
import matplotlib.pyplot as plt
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense, Lambda
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers.merge import concatenate as concat
from keras import backend as K
from keras.optimizers import Adam
%matplotlib inline


In [2]:
stock_data_file = 'data/all_stocks_5yr.csv'
stocks_df = pd.read_csv(stock_data_file, parse_dates=['date'])
stocks_df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [3]:
stocks_df['day'] = stocks_df['date'].dt.weekday
stocks_df['day_of_month'] = stocks_df['date'].dt.day
stocks_df.head()

,date,open,high,low,close,volume,Name,day,day_of_month
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,4,8
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,0,11
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,1,12
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,2,13
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,3,14


In [4]:
stock_symbols = list(set(stocks_df['Name']))

dow_30_list = ['NKE', 'AXP', 'BA', 'C', 'CAT', 'DD', 'DIS', 'GE', 'CSCO', 'HD', 'CVX', 'V', 'IBM',
'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'GS', 'MRK', 'MSFT', 'PFE', 'PG', 'AAPL', 'UTX', 'VZ',
'WMT', 'XOM', 'TRV']
 
dow_30_symbols = (list(set(dow_30_list) & set(stock_symbols)))

In [5]:
market_data = {}
for symbol in dow_30_symbols:
    # segregate data for that stock so it doesn't spill into other one
    temp_df = stocks_df[stocks_df['Name'] == symbol]
    open = temp_df['open']
    high = temp_df['high']
    low = temp_df['low']
    close = temp_df['close']
    volume = temp_df['volume']
    close_avg_3 =  temp_df['close'].rolling(3).mean()
    close_avg_5 =  temp_df['close'].rolling(5).mean()
    close_avg_10 =  temp_df['close'].rolling(10).mean()
    open_close = temp_df['close'] - temp_df['open']
    day_ = temp_df['day']
    day_of_month = temp_df['day_of_month']
    date = temp_df['date']
    # day of month
    
    market_data[symbol] = [open, high, low, close,
                                volume, close_avg_3, close_avg_5, close_avg_10, open_close, day_,
                               day_of_month, date]

In [6]:
# rebuild data frame
fin_open = []
fin_high = []
fin_low = []
fin_close = []
fin_volume = []
fin_close_avg_3 = []
fin_close_avg_5 = []
fin_close_avg_10 = []
fin_open_close = []

fin_day_ = []
fin_day_of_month = []
fin_date = []
fin_symbol = []

fin_high_high_diff3 = []
fin_low_low_diff3 = []
fin_open_open_diff3 = []
fin_close_close_diff3 = []
fin_volume_volume_diff3 = []


for key, value in market_data.items():
    fin_open.extend(list(value[0]))
    fin_high.extend(list(value[1]))
    fin_low.extend(list(value[2]))
    fin_close.extend(list(value[3]))
    fin_volume.extend(list(value[4]))
    fin_close_avg_3.extend(list(value[5]))
    fin_close_avg_5.extend(list(value[6]))
    fin_close_avg_10.extend(list(value[7]))
    fin_open_close.extend(list(value[8]))
    fin_day_.extend(list(value[9]))
    fin_day_of_month.extend(list(value[10]))
    fin_date.extend(list(value[11]))
    fin_symbol.extend([key] * len(value[11]))

# build final data frame for CVAE
norm_stocks_df = pd.DataFrame({'symbol':fin_symbol, 
                            'open':fin_open,
                            'high':fin_high,
                            'low':fin_low,
                            'close':fin_close,
                            'raw_close':fin_close,
                            'volume':fin_volume,
                            'raw_volume':fin_volume,
                            'close_avg_3':fin_close_avg_3,
                            'close_avg_5':fin_close_avg_5,
                            'close_avg_10':fin_close_avg_10,
                            'open_close':fin_open_close,
                            'raw_open_close':fin_open_close,
                            'day':fin_day_,
                            'day_of_month':fin_day_of_month,
                            'date':fin_date})

# remove any rows with NaN or inf columns
norm_stocks_df = norm_stocks_df.replace([np.inf, -np.inf], np.nan)
norm_stocks_df = norm_stocks_df.dropna(how='any')

norm_stocks_df.head(5)

,symbol,open,high,low,close,raw_close,volume,raw_volume,close_avg_3,close_avg_5,close_avg_10,open_close,raw_open_close,day,day_of_month,date
9,CSCO,20.81,20.930,20.72,20.900,20.900,20483295,20483295,20.921667,21.0430,21.0745,0.090,0.090,4,22,2013-02-22
10,CSCO,21.00,21.195,20.65,20.660,20.660,37362549,37362549,20.773333,20.9770,21.0245,-0.340,-0.340,0,25,2013-02-25
11,CSCO,20.54,20.780,20.50,20.608,20.608,34254647,34254647,20.722667,20.8066,20.9583,0.068,0.068,1,26,2013-02-26
12,CSCO,20.60,21.020,20.44,20.890,20.890,25189601,25189601,20.719333,20.7636,20.9503,0.290,0.290,2,27,2013-02-27
13,CSCO,20.89,21.045,20.74,20.855,20.855,30335762,30335762,20.784333,20.7826,20.9218,-0.035,-0.035,3,28,2013-02-28
